# benchmark different energy computation methods to see which one is the fastests

In [2]:
import timeit
import numpy as np

In [3]:
L = 8

In [4]:
state = np.random.rand(L**2)
new_state = lambda: np.random.rand(L**2)

number = 100_000
stime = np.array(timeit.repeat(lambda: np.random.rand(L**2), number=number))
nstime = np.array(timeit.repeat(lambda: new_state(), number=number))
print(f"{stime=} {nstime=} {nstime-stime}")

stime=array([0.175653 , 0.1215316, 0.1629311, 0.1770934, 0.1514511]) nstime=array([0.1582052, 0.1453214, 0.1418559, 0.2251198, 0.1636033]) [-0.0174478  0.0237898 -0.0210752  0.0480264  0.0121522]


In [14]:
one = np.array(timeit.repeat(lambda: (new_state() + new_state()) % 1.0, number=number))
twopi = np.array(timeit.repeat(lambda: (new_state() + new_state()) % (2*np.pi), number=number))
print(f"{one=} {twopi=} {twopi-one}")

one=array([0.6252415, 0.5893309, 0.5740458, 0.618112 , 0.5631939]) twopi=array([0.546328 , 0.5396909, 0.5791978, 0.7177333, 0.8828835]) [-0.0789135 -0.04964    0.005152   0.0996213  0.3196896]


In [30]:
def state_to_lattice(state):
    lattice = np.reshape(2 * np.pi * state, (L, L))
    return lattice


number = 1_000_000
np.array(timeit.repeat(lambda: state_to_lattice(new_state()), number=number)) / number

array([6.7391841e-06, 7.4477916e-06, 7.1829801e-06, 6.7562989e-06,
       6.2980928e-06])

In [59]:
def compute_energy(state):
    """
    Computes energy of the current state
    """
    # J=0 except for nearest neighbor
    lattice = state_to_lattice(state)
    energy = (
        -sum(
            np.cos(lattice[i, j] - lattice[i - 1, j])
            + np.cos(lattice[i, i] - lattice[i, j - 1])
            for i in range(L)
            for j in range(L)
        )
    ) / L**2
    return energy


number = 100_000
np.array(timeit.timeit(lambda: compute_energy(new_state()), number=number))

array(319.9243684)

In [ ]:
def compute_energy2(state):
    lattice = state_to_lattice(state)
    energy = (
        -np.sum(
            np.cos(lattice[i, j] - lattice[i - 1, j])
            + np.cos(lattice[i, i] - lattice[i, j - 1])
            for i in range(L)
            for j in range(L)
        )
        / L**2
    )
    return energy


number = 100_000
np.array(timeit.timeit(lambda: compute_energy2(new_state()), number=number))

In [61]:
def compute_energy_roll(state):
    lattice = state_to_lattice(state)
    energy = (
        -np.sum(
            np.cos(lattice - np.roll(lattice, -1, axis=0))
            + np.cos(lattice - np.roll(lattice, -1, axis=1))
        )
        / L**2
    )
    return energy


number = 100_000
np.array(timeit.timeit(lambda: compute_energy_roll(new_state()), number=number))

array(11.4378669)

In [72]:
number = 200_000
timeit.repeat(lambda: (new_state() + new_state()) % 1, number=number)

[1.240578099997947,
 1.3364106000080938,
 1.1644328999973368,
 1.1640502000082051,
 1.2120015999971656]